In [19]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
import pandas as pd
import yaml
import geopandas as gp
import time
import pandas as pd
from geopy.distance import great_circle
import folium
import h5py

In [20]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [21]:
with open('a-project.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project/output_FJSJ_16-01/'               # Harmon server
name_project

'project/output_regular_v9.5_5x13/'

In [22]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = dir_project_workspace + name_project
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project/output_regular_v9.5_5x13/


In [23]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [24]:
dir_CC = dir_CC_workspace + info_basic['rdir_CC']
dir_ds = dir_project + info_basic['rdir_ds']
dir_partition = dir_project + info_basic['rdir_partition']
dir_image = dir_project + info_basic['rdir_image'] + 'regular_FJ/'
if os.path.exists(dir_image) == False:
    os.makedirs(dir_image)

In [25]:
key_subworks = info_basic['key_subworks']
#key_subworks = ['07-03']
key_subworks

['07-03',
 '08-03',
 '09-03',
 '10-03',
 '11-03',
 '12-03',
 '13-03',
 '14-03',
 '15-03',
 '16-03',
 '17-03',
 '18-03',
 '19-03',
 '20-03',
 '21-03',
 '22-03',
 '23-03',
 '24-03',
 '25-03',
 '26-03',
 '27-03',
 '28-03',
 '29-03',
 '30-03',
 '31-03',
 '32-03',
 '33-03',
 '34-03',
 '35-03',
 '36-03',
 '37-03',
 '38-03',
 '39-03',
 '40-03',
 '41-03',
 '42-03',
 '43-03',
 '44-03',
 '45-03',
 '46-03',
 '47-03',
 '48-03',
 '49-03',
 '50-03',
 '51-03',
 '52-03',
 '53-03',
 '54-03',
 '07-04',
 '08-04',
 '09-04',
 '10-04',
 '11-04',
 '12-04',
 '13-04',
 '14-04',
 '15-04',
 '16-04',
 '17-04',
 '18-04',
 '19-04',
 '20-04',
 '21-04',
 '22-04',
 '23-04',
 '24-04',
 '25-04',
 '26-04',
 '27-04',
 '28-04',
 '29-04',
 '30-04',
 '31-04',
 '32-04',
 '33-04',
 '34-04',
 '35-04',
 '36-04',
 '37-04',
 '38-04',
 '39-04',
 '40-04',
 '41-04',
 '42-04',
 '43-04',
 '44-04',
 '45-04',
 '46-04',
 '47-04',
 '48-04',
 '49-04',
 '50-04',
 '51-04',
 '52-04',
 '53-04',
 '54-04',
 '07-05',
 '08-05',
 '09-05',
 '10-05',


### read all stations coordinates

In [26]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = mathlib.GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_stations_all = stainfo['latitude'].tolist() 
lon_stations_all = stainfo['longitude'].tolist()

In [27]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [28]:
ncffile = h5py.File(dir_CC + 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f = ncffile['f'][:]
count_all = ncffile['count'][:]
ncffile.close()

In [29]:
stations_this = {}
lat_stations_this = {}
lon_stations_this = {}
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations, lat, lon = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_this[key] = stations_this
    lat_stations_this[key] = lat.astype(float)
    lon_stations_this[key] = lon.astype(float)

In [30]:
def time_window_filter(t,ncfst0,r,v_min,t0,a):
    ncfst = ncfst0.copy()
    for i in range(len(ncfst)):
        tag = r[i]/v_min
        #print(t0,tag)
        t1 = t[t>-tag-t0][t[t>-tag-t0]< tag+t0]
        start = np.where(t == t1[0])[0][0]
        end = np.where(t == t1[-1])[0][0]
        ncfst[i][start:end+1]= ncfst[i][start:end+1]* np.exp(-a*np.abs((tag-np.abs(t1))))
    return ncfst

In [31]:
def remove_stack(key_subwork,ncfs_sum_linear,r,StationPairs):
    global dir_project
    global key_subworks
    global info_basic
    global flag_save
    global v_tag
    global t0
    global a

    start0 = time.time()
    f0 = info_basic_bi['f']
    dt = 1/np.max(f0)
    t = (np.linspace(-len(f0)-1,len(f0)-1,2*(len(f0)-1))+0.5)*dt/2
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    ncfst1 = time_window_filter(t,ncfst_linear,r,v_tag,t0,a)
    ncfs_sum_remove = mathlib.time_freq(ncfst1)
    if flag_save:
        tag = name_project[ name_project.rfind('_')+1: -1]
        dir_stack = dir_project + 'stack_'+tag+'/'
        outname = key_subwork+'_gather_timewindow.h5'
        if os.path.exists(dir_stack+outname):
            os.remove(dir_stack+outname)
        ncffile = h5py.File(dir_stack+outname,'w')
        ncffile.create_dataset('ncfs', data=ncfs_sum_remove) 
        ncffile.create_dataset('r',data=r)
        ncffile.create_dataset('StationPairs',data=StationPairs)
        ncffile.close()
    #print('Finish remove stack, time:', time.time()-start0, ' seconds')
    return ncfs_sum_remove

In [32]:
v_tag = info_basic['remove_v_tag']
t0 = info_basic['remove_t0']
a = info_basic['remove_a']

In [33]:
flag_save = 0

start = 0 
interval = 10
flag_time = 0.02
tao = 0

c_map = 'jet'
v_min = 0.1
v_max = None
d_len = 31

xlim_f = [3,30]
xlim_T = [-1,1]

In [34]:
ds_linear_all = {}
ds_remove_all = {}
index_ncfs = {}
rs = {}
for key_subwork in key_subworks:
    print(key_subwork+ '; '+str(key_subworks.index(key_subwork)+1)+'/'+str(len(key_subworks))+' subworks.')
    ds = h5py.File(dir_ds+'ds_'+str(key_subwork)+'.h5', 'r')
    ds_linear = plotlib.smooth_ds(ds['ds_linear'][0])
    ds_remove = plotlib.smooth_ds(ds['ds_remove'][0])
    ds_linear_all[key_subwork] = ds_linear
    ds_remove_all[key_subwork] = ds_remove
    index_ncfs[key_subwork] = list(ds['index_ncfs'])
    rs[key_subwork] = np.array(ds['r'])
    ds.close()

07-03; 1/768 subworks.
08-03; 2/768 subworks.
09-03; 3/768 subworks.
10-03; 4/768 subworks.
11-03; 5/768 subworks.
12-03; 6/768 subworks.
13-03; 7/768 subworks.
14-03; 8/768 subworks.
15-03; 9/768 subworks.
16-03; 10/768 subworks.
17-03; 11/768 subworks.
18-03; 12/768 subworks.
19-03; 13/768 subworks.
20-03; 14/768 subworks.
21-03; 15/768 subworks.
22-03; 16/768 subworks.
23-03; 17/768 subworks.
24-03; 18/768 subworks.
25-03; 19/768 subworks.
26-03; 20/768 subworks.
27-03; 21/768 subworks.
28-03; 22/768 subworks.
29-03; 23/768 subworks.
30-03; 24/768 subworks.
31-03; 25/768 subworks.
32-03; 26/768 subworks.
33-03; 27/768 subworks.
34-03; 28/768 subworks.
35-03; 29/768 subworks.
36-03; 30/768 subworks.
37-03; 31/768 subworks.
38-03; 32/768 subworks.
39-03; 33/768 subworks.
40-03; 34/768 subworks.
41-03; 35/768 subworks.
42-03; 36/768 subworks.
43-03; 37/768 subworks.
44-03; 38/768 subworks.
45-03; 39/768 subworks.
46-03; 40/768 subworks.
47-03; 41/768 subworks.
48-03; 42/768 subworks.
4

In [35]:
for key_subwork in key_subworks:
    print(key_subwork+ '; '+str(key_subworks.index(key_subwork)+1)+'/'+str(len(key_subworks))+' subworks.')
    index = index_ncfs[key_subwork]
    ncfs_sum_linear = np.array(ncfs)[index,:]
    r = rs[key_subwork]
    
    # plot area
    stalist = stations_this[key_subwork]
    lat = lat_stations_this[key_subwork].astype(float)
    lon = lon_stations_this[key_subwork].astype(float)
    nsta = len(stalist)
    StationPairs = mathlib.GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)

    ncfs_sum_remove = remove_stack(key_subwork,ncfs_sum_linear,r,StationPairs)

    fig,ax = plt.subplots(3,3,figsize= (22,18))
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_stations_all,lat_stations_all,lon,lat)
    for i in range(len(faults)):
        ax[0][0].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
    

    # plot ncfs and ncfst
    # linear stack
    f0 = info_basic_bi['f']
    t = info_basic_bi['t']
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    ncfst_remove = mathlib.freq_time(ncfs_sum_remove)
    
    title1 = "Linear Stack subarea frequency for "+key_subwork
    #xlim_f = [0,15]
    ax[0][1] = plotlib.plot_ncfs(ax[0][1],f0,ncfs_sum_linear,r,title1,xlim_f,0)
    ax[0][2] = plotlib.plot_ncfs(ax[0][2],f0,ncfs_sum_remove,r,title1,xlim_f,0)
    
    # plot fj
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_linear = ds['ds_linear'][0]
    ds_remove = ds['ds_remove'][0]
    ds_linear = plotlib.smooth_ds(ds_linear)
    ds_remove = plotlib.smooth_ds(ds_remove)
    c = np.linspace(info_basic['fj_c_min'],info_basic['fj_c_max'],info_basic['fj_c_num'])
    #c = np.linspace(100,2000,1000)
    title0 = "Linear Stack subarea time for "+key_subwork
    ax[1][0] = plotlib.plot_ncfst(ax[1][0],t,ncfst_linear[start::interval],r[start::interval],title0,flag_time,xlim_T,0)
    title0 = "Remove Stack subarea time for "+key_subwork
    ax[1][1] = plotlib.plot_ncfst(ax[1][1],t,ncfst_remove[start::interval],r[start::interval],title0,flag_time,xlim_T,0)
    for i in range(len(r)):
        ax[1][1].plot()


    
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[2][0] = plotlib.plot_fj(ax[2][0],ds_linear,title0,f0,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
    ax[2][0].set_xlim(xlim_f)

    title0 = "Remove stack dispersion curve "+str(d_len)+' days'
    ax[2][1] = plotlib.plot_fj(ax[2][1],ds_remove,title0,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
    ax[2][1].set_xlim(xlim_f)


    plt.tight_layout()
    plt.savefig(dir_image+key_subwork+'_compare_remove.png',dpi=100)
    plt.close()

07-03; 1/768 subworks.
08-03; 2/768 subworks.
09-03; 3/768 subworks.
10-03; 4/768 subworks.
11-03; 5/768 subworks.
12-03; 6/768 subworks.
13-03; 7/768 subworks.
14-03; 8/768 subworks.
15-03; 9/768 subworks.
16-03; 10/768 subworks.
17-03; 11/768 subworks.
18-03; 12/768 subworks.
19-03; 13/768 subworks.
20-03; 14/768 subworks.
21-03; 15/768 subworks.
22-03; 16/768 subworks.
23-03; 17/768 subworks.
24-03; 18/768 subworks.
25-03; 19/768 subworks.
26-03; 20/768 subworks.
27-03; 21/768 subworks.
28-03; 22/768 subworks.
29-03; 23/768 subworks.
30-03; 24/768 subworks.
31-03; 25/768 subworks.
32-03; 26/768 subworks.
33-03; 27/768 subworks.
34-03; 28/768 subworks.
35-03; 29/768 subworks.
36-03; 30/768 subworks.
37-03; 31/768 subworks.
38-03; 32/768 subworks.
39-03; 33/768 subworks.
40-03; 34/768 subworks.
41-03; 35/768 subworks.
42-03; 36/768 subworks.
43-03; 37/768 subworks.
44-03; 38/768 subworks.
45-03; 39/768 subworks.
46-03; 40/768 subworks.
47-03; 41/768 subworks.
48-03; 42/768 subworks.
4

In [36]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)